# HW 1

### Decision Variables
Let $x_{i j}$ be the number of units of product $i$ to be produced in week $j$, where $i=$ $1,2, \ldots, 100$ and $j=1,2, \ldots, 52$.

### Objective Function
Maximize:
Profit - Holding Cost $=\sum_{i=1}^{100} \sum_{j=1}^{52} x_{i j} \times$ profit $_i-\sum_{i=1}^{500} \sum_{j=1}^{52}$ inventory $_{i j} \times$ holding $_j$

Constraints
1. $x_{i j} \leq \operatorname{demand}_{i j}$ (cannot produce more than demand)
2. inventory $_{i j} \geq 0$ (inventory cannot be negative)
3. Material availability constraint:
$$
\text { inventory }_{i j}+\sum_{k=1}^{100} x_{k j} \times \text { requirements }_{i k} \leq \text { availability }_{i j}
$$

In [1]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("JuMP")
Pkg.add("Gurobi")

    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


   Resolving package versions...


  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [2]:
using CSV, DataFrames, JuMP, Gurobi

Row,materials,Product 1,Product 2,Product 3,Product 4,Product 5,Product 6,Product 7,Product 8,Product 9,Product 10,Product 11,Product 12,Product 13,Product 14,Product 15,Product 16,Product 17,Product 18,Product 19,Product 20,Product 21,Product 22,Product 23,Product 24,Product 25,Product 26,Product 27,Product 28,Product 29,Product 30,Product 31,Product 32,Product 33,Product 34,Product 35,Product 36,Product 37,Product 38,Product 39,Product 40,Product 41,Product 42,Product 43,Product 44,Product 45,Product 46,Product 47,Product 48,Product 49,Product 50,Product 51,Product 52,Product 53,Product 54,Product 55,Product 56,Product 57,Product 58,Product 59,Product 60,Product 61,Product 62,Product 63,Product 64,Product 65,Product 66,Product 67,Product 68,Product 69,Product 70,Product 71,Product 72,Product 73,Product 74,Product 75,Product 76,Product 77,Product 78,Product 79,Product 80,Product 81,Product 82,Product 83,Product 84,Product 85,Product 86,Product 87,Product 88,Product 89,Product 90,Product 91,Product 92,Product 93,Product 94,Product 95,Product 96,Product 97,Product 98,Product 99,⋯
,Int64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,⋯
1,1,0.0,25.57,0.0,0.0,0.0,0.0,50.66,27.97,47.69,26.48,41.29,0.0,48.72,35.57,0.0,0.0,0.0,0.0,50.78,0.0,0.0,0.0,35.69,0.0,0.0,0.0,38.45,0.0,0.0,33.2,0.0,29.9,0.0,0.0,64.48,0.0,52.9,0.0,0.0,0.0,0.0,0.0,32.27,45.57,50.99,43.34,0.0,0.0,0.0,0.0,27.68,42.11,0.0,0.0,51.44,49.32,0.0,0.0,33.86,30.92,0.0,61.42,0.0,0.0,43.88,42.19,0.0,0.0,44.47,0.0,39.06,27.42,39.6,0.0,0.0,0.0,0.0,0.0,0.0,45.88,0.0,0.0,0.0,0.0,73.81,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,52.19,51.67,0.0,⋯
2,2,0.0,0.0,0.0,0.0,46.12,41.95,0.0,0.0,27.5,0.0,0.0,36.98,0.0,0.0,0.0,44.12,46.94,0.0,42.74,0.0,0.0,0.0,61.24,0.0,33.86,48.11,0.0,0.0,0.0,0.0,45.49,0.0,0.0,54.82,0.0,28.89,0.0,0.0,0.0,43.03,0.0,0.0,0.0,0.0,31.42,0.0,28.16,0.0,0.0,0.0,16.08,0.0,0.0,27.5,0.0,0.0,45.68,0.0,0.0,0.0,46.46,22.91,0.0,44.84,33.64,0.0,0.0,55.26,33.02,39.03,41.39,0.0,0.0,0.0,0.0,19.16,45.2,0.0,0.0,0.0,32.61,0.0,0.0,65.43,23.4,37.54,42.57,40.33,0.0,0.0,21.11,48.43,0.0,0.0,0.0,0.0,0.0,0.0,17.03,⋯
3,3,0.0,38.74,0.0,0.0,0.0,0.0,0.0,0.0,18.97,0.0,39.48,0.0,51.95,40.4,0.0,0.0,48.69,0.0,36.52,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,72.56,0.0,0.0,0.0,36.68,0.0,0.0,0.0,50.39,0.0,37.48,0.0,0.0,0.0,0.0,0.0,0.0,33.5,0.0,0.0,36.14,0.0,34.04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,43.14,0.0,0.0,0.0,0.0,50.93,0.0,0.0,44.48,0.0,0.0,0.0,0.0,0.0,40.66,0.0,0.0,40.51,0.0,0.0,0.0,0.0,0.0,27.25,0.0,43.0,49.53,0.0,0.0,34.3,0.0,0.0,0.0,0.0,31.71,0.0,22.51,⋯
4,4,0.0,47.14,0.0,0.0,0.0,22.35,0.0,51.1,0.0,0.0,39.64,0.0,0.0,67.34,52.36,0.0,52.86,0.0,57.31,47.56,0.0,0.0,51.51,0.0,0.0,0.0,15.17,0.0,0.0,0.0,0.0,56.84,0.0,43.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,33.9,46.27,0.0,0.0,45.37,33.42,0.0,0.0,0.0,0.0,0.0,44.57,0.0,0.0,0.0,0.0,0.0,0.0,0.0,26.51,0.0,39.35,30.33,0.0,0.0,47.87,0.0,27.67,0.0,0.0,0.0,38.46,38.54,0.0,50.77,27.7,0.0,0.0,0.0,42.15,0.0,24.86,0.0,0.0,73.87,0.0,17.88,0.0,0.0,46.09,0.0,0.0,0.0,0.0,0.0,0.0,0.0,⋯
5,5,0.0,0.0,0.0,0.0,0.0,0.0,49.59,0.0,47.05,0.0,40.88,40.93,62.19,0.0,0.0,0.0,23.22,57.6,71.71,45.5,46.45,0.0,0.0,32.72,32.28,0.0,0.0,0.0,0.0,0.0,0.0,45.05,0.0,32.47,0.0,44.31,0.0,29.5,0.0,23.66,0.0,49.15,0.0,46.93,31.07,0.0,42.22,0.0,0.0,0.0,0.0,0

In [17]:
# Load data
availability = CSV.read("availability.csv", DataFrame)
demand = CSV.read("demand.csv", DataFrame)
holding = CSV.read("holding.csv", DataFrame)
profit = CSV.read("profit.csv", DataFrame)
requirements = CSV.read("requirements.csv", DataFrame)

num_products = 100
num_weeks = 52
num_materials = 500

# Initialize the model
model = Model(Gurobi.Optimizer)

# Decision variables
@variable(model, x[1:num_products, 1:num_weeks] >= 0)
@variable(model, inventory[1:num_materials, 1:num_weeks] >= 0)

# Objective function
@objective(model, Max, sum(x[i, j] * profit[i, :][1] for i in 1:num_products, j in 1:num_weeks) -
                      sum(inventory[i, j] * holding[j, :][1] for i in 1:num_materials, j in 1:num_weeks))

# Demand constraints
for i in 1:num_products
    for j in 1:num_weeks
        @constraint(model, x[i, j] <= demand[i, j])
    end
end

# Material availability constraints
for i in 1:num_materials
    for j in 1:num_weeks
        @constraint(model, inventory[i, j] + sum(x[k, j] * requirements[i, k] for k in 1:num_products) <= availability[i, j])
    end
end

# Solve the model
optimize!(model)

# Extract results
total_profit = objective_value(model)
x_values = value.(x)
inventory_values = value.(inventory)

# Calculate total holding cost
total_holding_cost = sum(inventory_values[i, j] * holding[j, :][1] for i in 1:num_materials, j in 1:num_weeks)

# Print results
println("Total Profit: ", total_profit)
println("Total Holding Cost: ", total_holding_cost)

Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-15
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (mac64[arm])

CPU model: Apple M1 Max
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads

Optimize a model with 31200 rows, 31200 columns and 902044 nonzeros
Model fingerprint: 0xf41b08d4
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [1e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 3e+05]
Presolve removed 11358 rows and 26000 columns
Presolve time: 0.22s
Presolved: 5200 rows, 20814 columns, 673609 nonzeros

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 2.574e+05
 Factor NZ  : 2.626e+05 (roughly 13 MB of memory)
 Factor Ops : 1.759e+07 (less than 1 second per iteration)
 Threads    : 8

Barrier performed 0 iterations in 0.34 seconds (0.66 work units)
Barrier so